In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as func

spark = SparkSession.builder.getOrCreate()

## Caderno de teste

### Construindo DataFrames e Criando arquivos parquet

In [2]:
dados= [(1, 3545, 3000, 6.99),
    (2, 3545, 4500, 0.45),
    (3,3509, 69998, 0.0),
    (4, 3510, 1, None ),
    (5, 4510, 34, 40.00)
    ]

schema = StructType([\
    StructField("transaction_id", IntegerType(), True),\
    StructField("client_id", IntegerType(), True),\
    StructField("total_amount", IntegerType(), True),\
    StructField("discount_percentage", FloatType())\
    ])

Transacoes = spark.createDataFrame(data=dados, schema=schema)

In [3]:
dados2 = [(3,3545, "Magazine Luana", 2.00, "True" ),
    (4, 3545,"Magazine Luana", 1.95, "False"), 
    (5, 3509, "Lojas Italianas", 1.00, "True"),
    (6, 3510, "Carrerfive", 3.00, "True")]
    
schema2 = StructType([\
    StructField("contract_id", IntegerType(), True),\
    StructField("client_id", IntegerType(), True),\
    StructField("client_name", StringType(), True),\
    StructField("percentage", FloatType(), True),\
    StructField("is_active",StringType(),True)
    ])
contrato = spark.createDataFrame(data=dados2, schema=schema2)

In [ ]:
df = spark.read.parquet('contrato.parquet')

In [ ]:
Transacoes.write.parquet("transacoes.parquet/")
contrato.write.parquet("contrato.parquet/")

### PySpark 

In [34]:
def ganho_total(transacao, contrato):
    desafio = contrato.join(Transacoes, contrato.client_id == Transacoes.client_id, 'left')\
        .filter(contrato.is_active == True)\
        .select(func.round(sum((col('total_amount')-(col('total_amount')*coalesce(col('discount_percentage'),lit(0.0))/100))*col('percentage'))/100,2).alias('mes')\
        .alias('Fechamento'))
    desafio.write.parquet('/home/desafio3.parquet/')
    return desafio
    
ganho_total(Transacoes, contrato)

+----------+
|Fechamento|
+----------+
|    845.41|
+----------+



### Pyspark com SQL

In [ ]:
Transacoes.createOrReplaceTempView('Transacoes')
contrato.createOrReplaceTempView('contrato')
Transacoes.show()
contrato.show()

In [31]:
teste3 = spark.sql('''
    select
        round(sum((T.total_amount-((T.total_amount*coalesce(T.discount_percentage,0)) / 100)) * (c.percentage) / 100),2) as Mes  
    from 
        contrato as c
        left join Transacoes as T on c.client_id = T.client_id
    where is_active = 'True'
    ''')
teste3.show()

+------+
|   Mes|
+------+
|845.41|
+------+

